*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.3
IPython 6.2.1

torch 0.3.0.post4


# Model Zoo -- Convolutional Neural Network

In [2]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import numpy as np


##########################
### SETTINGS
##########################

# Hyperparameters
learning_rate = 0.05
num_epochs = 10
batch_size = 128

# Architecture
num_features = 784
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [3]:
##########################
### MODEL
##########################

def same_padding(kernel_width, stride):
    return (kernel_width - stride) // 2


class ConvNet(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(ConvNet, self).__init__()
        
        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=same_padding(3, 1))
        # 28x28x4 => 14x14x4                              
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=same_padding(2, 2))                                       
        # 14x14x4 => 14x14x8
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=same_padding(3, 1))                 
        # 14x14x8 => 7x7x8                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=same_padding(2, 1))
        
        self.linear_1 = torch.nn.Linear(7*7*8, num_classes)

        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        logits = self.linear_1(out.view(-1, 7*7*8))
        probas = F.softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(0)
model = ConvNet(num_features=num_features,
                num_classes=num_classes)

if torch.cuda.is_available():
    model.cuda()
    

##########################
### COST AND OPTIMIZER
##########################

cost_fn = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [4]:
from torch.autograd import Variable
import torch.nn.functional as F

def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = Variable(features)
        if torch.cuda.is_available():
            features = features.cuda()
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas.data, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels.cpu() == targets).sum()
    return correct_pred/num_examples * 100
    

for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = Variable(features)
        targets = Variable(targets)
        
        if torch.cuda.is_available():
            features, targets = features.cuda(), targets.cuda()
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost.data[0]))

    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/468 | Cost: 2.3068
Epoch: 001/010 | Batch 050/468 | Cost: 2.2796
Epoch: 001/010 | Batch 100/468 | Cost: 1.7151
Epoch: 001/010 | Batch 150/468 | Cost: 0.7436
Epoch: 001/010 | Batch 200/468 | Cost: 0.6053
Epoch: 001/010 | Batch 250/468 | Cost: 0.2372
Epoch: 001/010 | Batch 300/468 | Cost: 0.2800
Epoch: 001/010 | Batch 350/468 | Cost: 0.3217
Epoch: 001/010 | Batch 400/468 | Cost: 0.4143
Epoch: 001/010 | Batch 450/468 | Cost: 0.1395
Epoch: 001/010 training accuracy: 91.98%
Epoch: 002/010 | Batch 000/468 | Cost: 0.1391
Epoch: 002/010 | Batch 050/468 | Cost: 0.2119
Epoch: 002/010 | Batch 100/468 | Cost: 0.2352
Epoch: 002/010 | Batch 150/468 | Cost: 0.1575
Epoch: 002/010 | Batch 200/468 | Cost: 0.1820
Epoch: 002/010 | Batch 250/468 | Cost: 0.1450
Epoch: 002/010 | Batch 300/468 | Cost: 0.1410
Epoch: 002/010 | Batch 350/468 | Cost: 0.2163
Epoch: 002/010 | Batch 400/468 | Cost: 0.1531
Epoch: 002/010 | Batch 450/468 | Cost: 0.1534
Epoch: 002/010 training accuracy: 94.56

In [5]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.55%
